In [1]:
#------------------------------------#
#------------- packages--------------#
#------------------------------------#

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date

import requests

import os

In [2]:
pids = pd.read_csv('StudyParticipants.csv')
aware_ids = pd.read_csv('aware_device.csv')

# merge the two dataframes on ParticipantID and keep only the rows from pids
pids = pd.merge(pids, aware_ids, on='ParticipantID', how='left')
pids

,ParticipantID,Phone,Awair,AwairID,Work,device_id,brand,build_id,manufacturer,model
0,edr,iPhone,AW-13145,70886b1232f6,Link Lab,393b2587-70fc-4e41-9368-7a14648375e3,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
1,egl,iPhone,AW-13145,70886b1232f6,Link Lab,a268bf35-24a7-40eb-a1fc-94142ffd3e58,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
2,j02,iPhone,AW-25396,70886b125db1,Off Grounds BR,9e61c8ae-5a4f-41b0-80ba-08ab06d495c6,iPhone,Darwin Kernel Version 21.3.0: Wed Jan 5 21:44...,Apple,iPhone
3,uja,iPhone,AW-16515,70886b123935,Link Lab,0dd46ec9-e40a-4791-9cee-3058dadc1959,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
4,lpz,Android,AW-29799,70886b127d5c,Off Grounds BR,5f863ec2-9314-41eb-8425-bf7173194f00,samsung,SP1A.210812.016.A526U1UES5CVD1,Samsung,SM-A526U1
5,oyb,Android,AW-23619,70886b126976,Link Lab,18519687-7d50-4493-8bdb-309d4d29dfd8,google,SP2A.220305.012,Google,Pixel 4a
6,mr1,iPhone,AW-23810,70886b126a7b,Link Lab,78004874-9c43-4d4f-8962-2db67c6ffe4a,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
7,xil,Android,AW-22996,70886b12532b,Link Lab,08d621fd-54ca-43de-9102-46bb2a17cce6,google,SP2A.220505.002,Google,Pixel 4a
8,pgm,iPhone,AW-15886,70886b123bc8,Link Lab,f7a16ee5-b15c-4034-ba52-4becf44cc74b,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
9,h9u,iPhone,AW-27185,70886b12697a,Off Grounds,04346571-51a9-4272-b6b1-52ac48248515,iPhone,Darwin Kernel Version 21.3.0: Wed Jan 5 21:44...,Apple,iPhone


In [3]:
aware = []

for r,d,f in os.walk("/Users/beatriceli/Documents/PhD_Research/GitHub/well-being/lll_aware"):
    for file in f:
        if file.endswith("calls.csv"):
            temp = pd.read_csv(os.path.join(r,file),parse_dates=["device_id"])
            aware.append(temp)

aware = pd.concat(aware)
# create columns called "ParticipantID" and "Work" and get value from matching device_id in aware to device_id in pids
# if device_ide in aware matches device_id in pids, then get the value of ParticipantID and Work from pids
aware = pd.merge(aware, pids[['device_id', 'ParticipantID', 'Work']], on='device_id', how='left')
# rename device_id.1 to screen_status
aware = aware.rename(columns={'device_id.1': 'screen_status'})
# convert epoch time to datetime and round to nearest second
aware['timestamp'] = pd.to_datetime(aware['timestamp'], unit='ms').dt.round('s')
# drop _id, device_id, and call_duration.1 columns
aware = aware.drop(['_id', 'device_id', 'call_duration.1'], axis=1)
# create date column
aware['date'] = aware['timestamp'].dt.date
# reorder columns: ParticipantID, Work, timestamp, call_type, call_duration
aware = aware[['ParticipantID', 'Work', 'date','timestamp', 'call_type', 'call_duration']]
# sort by ParticipantID and timestamp
aware = aware.sort_values(by=['ParticipantID', 'timestamp']).reset_index(drop=True)
aware.head()

,ParticipantID,Work,date,timestamp,call_type,call_duration
0,edr,Link Lab,2022-04-27,2022-04-27 19:48:01,1,0
1,edr,Link Lab,2022-04-27,2022-04-27 19:48:20,4,18
2,edr,Link Lab,2022-04-28,2022-04-28 22:29:17,1,0
3,edr,Link Lab,2022-04-28,2022-04-28 22:29:37,2,19
4,edr,Link Lab,2022-04-28,2022-04-28 22:30:12,4,35


In [5]:
# NA values
aware.isna().sum()

ParticipantID    0
Work             0
date             0
timestamp        0
call_type        0
call_duration    0
dtype: int64

In [6]:
# get first and last timestamp for each ParticipantID
aware.groupby('ParticipantID')['timestamp'].agg(['min', 'max'])

,min,max
ParticipantID,,
edr,2022-04-27 19:48:01,2022-07-23 21:00:33
egl,2022-04-27 15:58:43,2022-06-29 20:14:13
h9u,2022-05-05 20:06:28,2022-07-24 16:01:55
j02,2022-04-28 19:15:42,2022-06-17 17:58:33
mr1,2022-05-05 01:03:40,2022-07-07 21:39:56
pgm,2022-05-06 20:28:30,2022-06-27 22:47:23
uja,2022-04-26 20:57:36,2022-06-30 20:26:55
vxx,2022-05-06 14:54:22,2022-06-30 14:47:10


none of the android users have calls (only 1 wfh participant)

In [7]:
# create time_start and time_end columns with respect to each ParticipantID
for p in aware['ParticipantID'].unique():
    # time_end is the timestamp + call_duration
    aware.loc[aware['ParticipantID'] == p, 'time_start'] = aware.loc[aware['ParticipantID'] == p, 'timestamp'] - pd.to_timedelta(aware.loc[aware['ParticipantID'] == p, 'call_duration'], unit='s')
    # time_end is the timestamp
    aware.loc[aware['ParticipantID'] == p, 'time_end'] = aware.loc[aware['ParticipantID'] == p, 'timestamp']

# drop timestamp
aware = aware.drop(['timestamp'], axis=1)
aware.head()

,ParticipantID,Work,date,call_type,call_duration,time_start,time_end
0,edr,Link Lab,2022-04-27,1,0,2022-04-27 19:48:01,2022-04-27 19:48:01
1,edr,Link Lab,2022-04-27,4,18,2022-04-27 19:48:02,2022-04-27 19:48:20
2,edr,Link Lab,2022-04-28,1,0,2022-04-28 22:29:17,2022-04-28 22:29:17
3,edr,Link Lab,2022-04-28,2,19,2022-04-28 22:29:18,2022-04-28 22:29:37
4,edr,Link Lab,2022-04-28,4,35,2022-04-28 22:29:37,2022-04-28 22:30:12


In [8]:
# max call_duration for call_type = 1 (incoming calls)
aware.loc[aware['call_type'] == 1, 'call_duration'].max()

0

In [9]:
# max call_duration for call_type = 3 (dialing)
aware.loc[aware['call_type'] == 3, 'call_duration'].max()

0

In [10]:
# min max mean call_duration for call_type = 2 (connected)
aware.loc[aware['call_type'] == 2, 'call_duration'].agg(['min', 'max', 'mean'])

min      0.000000
max     44.000000
mean     8.982032
Name: call_duration, dtype: float64

In [11]:
# min max mean call_duration for call_type = 3 (dialing)
aware.loc[aware['call_type'] == 3, 'call_duration'].agg(['min', 'max', 'mean'])

min     0.0
max     0.0
mean    0.0
Name: call_duration, dtype: float64

In [12]:
# min max mean call_duration for call_type = 4 (disconnected)
aware.loc[aware['call_type'] == 4, 'call_duration'].agg(['min', 'max', 'mean'])

min        0.000000
max     9084.000000
mean     207.088033
Name: call_duration, dtype: float64

In [13]:
# cast call_type as int, ios and android have different call_type values
# 1=incoming, 2=connected, 3=dialing, 4=disconnected
aware['call_type'] = aware['call_type'].astype(int)
aware['call_type'] = aware['call_type'].replace({1: 'incoming', 2: 'connected', 3: 'dialing', 4: 'disconnected'})

In [14]:
# create a pivot table index on ParticipantID and date, columns on call_type, values on call_duration
aware_pivot = pd.pivot_table(aware, index=['ParticipantID', 'Work','date'], columns='call_type', values='call_duration', aggfunc=['count','sum']).reset_index()
# flatten the pivot table
aware_pivot.columns = ['_'.join(col).strip() for col in aware_pivot.columns]
aware_pivot = aware_pivot.reset_index(drop=True)
# fill NA values with 0
aware_pivot = aware_pivot.fillna(0)
aware_pivot.head()


,ParticipantID_,Work_,date_,count_connected,count_dialing,count_disconnected,count_incoming,sum_connected,sum_dialing,sum_disconnected,sum_incoming
0,edr,Link Lab,2022-04-27,0.0,0.0,1.0,1.0,0.0,0.0,18.0,0.0
1,edr,Link Lab,2022-04-28,1.0,0.0,1.0,1.0,19.0,0.0,35.0,0.0
2,edr,Link Lab,2022-04-29,2.0,3.0,3.0,0.0,15.0,0.0,122.0,0.0
3,edr,Link Lab,2022-04-30,5.0,3.0,5.0,2.0,24.0,0.0,763.0,0.0
4,edr,Link Lab,2022-05-01,4.0,3.0,4.0,1.0,21.0,0.0,246.0,0.0


In [15]:
# rename ParticipantID_ to ParticipantID, Work_ to Work, and date_ to date
aware_pivot = aware_pivot.rename(columns={'ParticipantID_': 'ParticipantID', 'Work_': 'Work', 'date_': 'date'})
# cast columns to int except for ParticipantID, Work, and date (after index 2)
aware_pivot.iloc[:,3:] = aware_pivot.iloc[:,3:].astype(int)
# drop sum_incoming, sum_dialing
aware_pivot = aware_pivot.drop(['sum_incoming', 'sum_dialing'], axis=1)


/var/folders/w9/kdq4wyw97jn4cg1bhrvrq8j80000gn/T/ipykernel_3890/4188124098.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  aware_pivot.iloc[:,3:] = aware_pivot.iloc[:,3:].astype(int)


In [16]:
# to csv
aware_pivot.to_csv('aware_calls.csv', index=False)